In [1]:
## import google storage password key
import os

from datetime import datetime
from datetime import date
import sys
sys.path.append("/content/drive/My Drive/EE-Waternamics/Modele_Consommation_jour")
import imp
import Function_sous_comptage_func 
imp.reload(Function_sous_comptage_func)


import pandas as pd

opus_file = "opus/opus_narbonne.csv"
eveler_file = "eveler/narbonne_ville.csv"
lerne_file_1 = "lerne/narbonne_ville_extrait_Lerne_avec_unite_23052018_04092018.csv"
lerne_file_2 = "lerne/lerne_narbonne_01102019_19022020.csv"
lerne_file_3 = "lerne/narbonne_ville_extrait_Lerne_23052018_04092018_test.csv"


json_key  = '/home/david_mouquet/sous_comptage_tensorflowp/gbl-ist-veri-waternamics-b18b5f179b6a.json'
site      = "narbonne"


ModuleNotFoundError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [0]:

#### DONNEES OPUS ####
imp.reload(ssf)

df_opus = ssf.Read_OPUS_file_brut(site, opus_file , json_key)

df_opus = df_opus.query('`GRANDEUR_PTMES` in ["Ch_DCO__EB_SYS__",\
         "Ch_DCO__EE_SYS__",\
    "Ch_MEST__EB_SYS__",\
"Ch_MEST__EE_SYS__",\
"Ch_NGL__EB_SYS__",\
"Ch_NGL__EE_SYS__",\
"QJ__EB_SYS__",\
"QJ__EB_TOT__",\
"T__EE__",\
"QJ__EE_SYS__"]')



df_opus.drop_duplicates(inplace=True)
df_opus['label'] = df_opus.apply(lambda x: x['GRANDEUR_PTMES'] + x['UNITE'], axis=1)

df_opus = df_opus[['label','DATE','VALEUR']]
df_opus.sort_values(by=['DATE'],inplace=True)
df_opus.index = range(len(df_opus))
df_opus = df_opus.pivot_table(index='DATE',columns='label',values='VALEUR')
df_opus['Date_Str'] = df_opus.index

def set_time_utc(d):    
    return datetime.strptime(d, '%Y-%m-%d %H:%M:%S %Z')

def set_time_locale(d):   
    return d.tz_localize('UTC').tz_convert('Europe/Paris').replace(tzinfo=None)



df_opus['Date_UTC']    =  df_opus['Date_Str'].apply(set_time_utc)   
df_opus['Date_Locale'] =  df_opus['Date_UTC'].apply(set_time_locale)   
df_opus.set_index('Date_Locale',inplace=True)

df_opus.drop(columns=['Date_Str'],inplace=True)

df_opus['Ch_DCO_traitee']  = df_opus['Ch_DCO__EB_SYS__kg/j'] - df_opus['Ch_DCO__EE_SYS__kg/j']  
df_opus['Ch_MEST_traitee'] = df_opus['Ch_MEST__EB_SYS__kg/j'] - df_opus['Ch_MEST__EE_SYS__kg/j'] 
df_opus['Ch_NGL_traitee']  = df_opus['Ch_NGL__EB_SYS__kg/j'] - df_opus['Ch_NGL__EE_SYS__kg/j'] 

new_lab = ['OPUS_' + l for l in df_opus.columns]
df_opus.columns = new_lab


In [0]:
#### DONNEES EVELER ####

imp.reload(ssf)

df_eveler = ssf.Read_Eveler_file(site, eveler_file , json_key)


def set_time_locale(d):   
    return d.tz_convert('Europe/Paris').replace(tzinfo=None)

df_eveler['Date_UTC']    =  df_eveler.index  
df_eveler['Date_Locale'] =  df_eveler['Date_UTC'].apply(set_time_locale)   
df_eveler.set_index('Date_Locale',inplace=True)


P = df_eveler['P_ACT_KW']
Pmean_max = P.resample('D').max().mean()
Pmean_min = P.resample('D').min().mean()
removed_outliers = P.between(0.5*Pmean_min, 1.5*Pmean_max)

df_eveler_clean = df_eveler[removed_outliers]
df_conso_jour = 24*df_eveler_clean.resample('D').mean()

new_lab = ['EVELER_' + l for l in df_conso_jour.columns]
df_conso_jour.columns = new_lab


In [0]:
#### DONNEES LERNE ####
imp.reload(ssf)

df_lerne_1 = ssf.Read_Lerne_file_Formated(site, lerne_file_1 , json_key, ";", "%Y-%m-%d %H:%M:%S")
#df_dup = df_lerne[df_lerne[['mesure_date_val','equipement_nom']].duplicated()].sort_values(by=['mesure_date_val','equipement_nom'])

def set_time_locale(d):   
    return d.tz_localize('UTC').tz_convert('Europe/Paris').replace(tzinfo=None)

df_lerne_1['Date_UTC']    =  df_lerne_1.index  
df_lerne_1['Date_Locale'] =  df_lerne_1['Date_UTC'].apply(set_time_locale)   
df_lerne_1.set_index('Date_Locale',inplace=True)
l_status_1 = [l for l in df_lerne_1.columns if l.startswith('M_')]
df_lerne_1 = df_lerne_1[l_status_1]


Data/lerne/narbonne_ville_extrait_Lerne_082018_082019.csv


In [0]:
#### DONNEES LERNE ####
imp.reload(ssf)

df_lerne_2 = ssf.Read_Lerne_file(site, lerne_file_2 , json_key,",")
#df_dup = df_lerne[df_lerne[['mesure_date_val','equipement_nom']].duplicated()].sort_values(by=['mesure_date_val','equipement_nom'])

def set_time_locale(d):   
    return d.tz_convert('Europe/Paris').replace(tzinfo=None)

df_lerne_2['Date_UTC']    =  df_lerne_2.index  
df_lerne_2['Date_Locale'] =  df_lerne_2['Date_UTC'].apply(set_time_locale)   
df_lerne_2.set_index('Date_Locale',inplace=True)
l_status_2 = [l for l in df_lerne_2.columns if l.startswith('M_')]
df_lerne_2 = df_lerne_2[l_status_2]

In [0]:
#### DONNEES LERNE ####
imp.reload(ssf)

df_lerne_3 = ssf.Read_Lerne_file_Formated(site, lerne_file_3 , json_key, ";", "%Y-%m-%d %H:%M:%S")
#df_dup = df_lerne[df_lerne[['mesure_date_val','equipement_nom']].duplicated()].sort_values(by=['mesure_date_val','equipement_nom'])

def set_time_locale(d):   
    return d.tz_localize('UTC').tz_convert('Europe/Paris').replace(tzinfo=None)

df_lerne_3['Date_UTC']    =  df_lerne_3.index  
df_lerne_3['Date_Locale'] =  df_lerne_3['Date_UTC'].apply(set_time_locale)   
df_lerne_3.set_index('Date_Locale',inplace=True)
l_status_3 = [l for l in df_lerne_3.columns if l.startswith('M_')]
df_lerne_3 = df_lerne_3[l_status_3]



Data/lerne/narbonne_ville_extrait_Lerne_23052018_04092018_test.csv


In [0]:

Periode = '24H' # Fréquence d'aggrégation des résultats

# Temps de fonctionnement en heure
Equip_Work_Time_Equ_1 = ssf.Compute_Equi_TDM_Lerne(df_lerne_1, l_status_1, periode = Periode)
Equip_Work_Time_Equ_1['TDM_Total_Equ'] = Equip_Work_Time_Equ_1.sum(axis=1)

new_lab = ['LERNE_' + l for l in Equip_Work_Time_Equ_1.columns]
Equip_Work_Time_Equ_1.columns = new_lab


# Temps de fonctionnement en heure
Equip_Work_Time_Equ_2 = ssf.Compute_Equi_TDM_Lerne(df_lerne_2, l_status_2, periode = Periode)
Equip_Work_Time_Equ_2['TDM_Total_Equ'] = Equip_Work_Time_Equ_2.sum(axis=1)

new_lab = ['LERNE_' + l for l in Equip_Work_Time_Equ_2.columns]
Equip_Work_Time_Equ_2.columns = new_lab

# Temps de fonctionnement en heure
Equip_Work_Time_Equ_3 = ssf.Compute_Equi_TDM_Lerne(df_lerne_3, l_status_3, periode = Periode)
Equip_Work_Time_Equ_3['TDM_Total_Equ'] = Equip_Work_Time_Equ_3.sum(axis=1)

new_lab = ['LERNE_' + l for l in Equip_Work_Time_Equ_3.columns]
Equip_Work_Time_Equ_3.columns = new_lab


Equip_Work_Time_Equ_1.drop([Equip_Work_Time_Equ_1.index[0],Equip_Work_Time_Equ_1.index[-1]],inplace=True)
Equip_Work_Time_Equ_2.drop([Equip_Work_Time_Equ_2.index[0],Equip_Work_Time_Equ_2.index[-1]],inplace=True)
Equip_Work_Time_Equ_3.drop([Equip_Work_Time_Equ_3.index[0],Equip_Work_Time_Equ_3.index[-1]],inplace=True)
Equip_Work_Time_Equ = pd.concat([Equip_Work_Time_Equ_1,Equip_Work_Time_Equ_2, Equip_Work_Time_Equ_3])
Equip_Work_Time_Equ.fillna(0, inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [0]:

df_conso_jour.drop([df_conso_jour.index[0],df_conso_jour.index[-1]],inplace=True)



data = df_conso_jour.merge(Equip_Work_Time_Equ, how='outer', left_index = True, right_index = True)
data = data.merge(df_opus, how='outer', left_index = True, right_index = True)


data['EVELER_Conso/QJ__EB_TOT'] = data['EVELER_P_ACT_KW']/data['OPUS_QJ__EB_SYS__m³/j']
data['EVELER_Conso/DCO_traitee'] = data['EVELER_P_ACT_KW']/data['OPUS_Ch_DCO_traitee']



data.index.name='Date'

In [0]:
data

,EVELER_P_ACT_KW,EVELER_P_REACT_KVAR_POSITIVE,LERNE_M_AGITATEUR_1_ANAEROBIE_FILE_2,LERNE_M_AGITATEUR_1_ANOXIE_FILE_1,LERNE_M_AGITATEUR_1_ANOXIE_FILE_2,LERNE_M_AGITATEUR_1_OXYDATION_FILE_1,LERNE_M_AGITATEUR_1_OXYDATION_FILE_2,LERNE_M_AGITATEUR_2_ANAEROBIE_FILE_1,LERNE_M_AGITATEUR_2_OXYDATION_FILE_1,LERNE_M_AGITATEUR_2_OXYDATION_FILE_2,...,OPUS_Ch_NGL__EB_SYS__kg/j,OPUS_Ch_NGL__EE_SYS__kg/j,OPUS_QJ__EB_SYS__m³/j,OPUS_QJ__EE_SYS__m³/j,OPUS_Date_UTC,OPUS_Ch_DCO_traitee,OPUS_Ch_MEST_traitee,OPUS_Ch_NGL_traitee,EVELER_Conso/QJ__EB_TOT,EVELER_Conso/DCO_traitee
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6809.50,6789.0,2010-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN
2011-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,544.00256,34.14867,6789.00,6789.0,2011-01-01 23:00:00,5492.3010,2883.9672,509.85389,NaN,NaN
2011-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7844.38,7673.0,2011-01-02 23:00:00,NaN,NaN,NaN,NaN,NaN
2011-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7819.10,7587.0,2011-01-03 23:00:00,6141.6916,2754.9271,NaN,NaN,NaN
2011-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8885.84,8870.0,2011-01-04 23:00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-14,NaN,NaN,23.996667,24.0,24.0,11.647222,23.996667,23.996667,23.996667,0.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2020-02-15,NaN,NaN,23.997500,24.0,24.0,12.333333,23.997500,23.997500,11.664167,0.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2020-02-16,NaN,NaN,23.997500,24.0,24.0,23.997500,23.997500,23.997500,12.333333,0.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [0]:
data.columns

Index(['EVELER_P_ACT_KW', 'EVELER_P_REACT_KVAR_POSITIVE',
       'LERNE_AGITATEUR_001', 'LERNE_AGITATEUR_104', 'LERNE_AGITATEUR_105',
       'LERNE_AGITATEUR_204', 'LERNE_AGITATEUR_205', 'LERNE_AGITATEUR_BIOLIX',
       'LERNE_AGITATEUR_CUVE_RECEPTION', 'LERNE_AGITATEUR_MURISS_POLYMERE',
       'LERNE_BROSSE_101_CHENAL_1', 'LERNE_BROSSE_102_CHENAL_1',
       'LERNE_BROSSE_103_CHENAL_1', 'LERNE_BROSSE_201_CHENAL_2',
       'LERNE_BROSSE_202_CHENAL_2', 'LERNE_BROSSE_203_CHENAL_2',
       'LERNE_CLASSIFICATEUR', 'LERNE_DEGEL_PISTE_PONT_CLARIF_1',
       'LERNE_DEGEL_PISTE_PONT_CLARIF_2', 'LERNE_DEGRILLEUR_100',
       'LERNE_DEGRILLEUR_200', 'LERNE_DISPO_RESERVE_1',
       'LERNE_DISPO_RESERVE_2', 'LERNE_HYDROEJECTEUR_1_BASSIN',
       'LERNE_HYDROEJECTEUR_3_BASSIN', 'LERNE_HYDROEJECTEUR_MATIERES',
       'LERNE_MALAXEUR_BOUE_CHAUX', 'LERNE_MOTEUR_BOL_CENTRI_1',
       'LERNE_MOTEUR_BOL_CENTRI_2', 'LERNE_MOTEUR_VIS_CENTRI_1',
       'LERNE_MOTEUR_VIS_CENTRI_2', 'LERNE_P1_BACHE_TOUTES_EAUX

In [0]:
data.to_csv("Data/Daily/narbonne_opus_tdm_eveler_daily.csv")

In [0]:
data

,EVELER_P_ACT_KW,EVELER_P_REACT_KVAR_POSITIVE,LERNE_M_AGITATEUR_1_ANAEROBIE_FILE_2,LERNE_M_AGITATEUR_1_ANOXIE_FILE_1,LERNE_M_AGITATEUR_1_ANOXIE_FILE_2,LERNE_M_AGITATEUR_1_OXYDATION_FILE_1,LERNE_M_AGITATEUR_1_OXYDATION_FILE_2,LERNE_M_AGITATEUR_2_ANAEROBIE_FILE_1,LERNE_M_AGITATEUR_2_OXYDATION_FILE_1,LERNE_M_AGITATEUR_2_OXYDATION_FILE_2,...,OPUS_Ch_NGL__EB_SYS__kg/j,OPUS_Ch_NGL__EE_SYS__kg/j,OPUS_QJ__EB_SYS__m³/j,OPUS_QJ__EE_SYS__m³/j,OPUS_Date_UTC,OPUS_Ch_DCO_traitee,OPUS_Ch_MEST_traitee,OPUS_Ch_NGL_traitee,EVELER_Conso/QJ__EB_TOT,EVELER_Conso/DCO_traitee
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6809.50,6789.0,2010-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN
2011-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,544.00256,34.14867,6789.00,6789.0,2011-01-01 23:00:00,5492.3010,2883.9672,509.85389,NaN,NaN
2011-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7844.38,7673.0,2011-01-02 23:00:00,NaN,NaN,NaN,NaN,NaN
2011-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7819.10,7587.0,2011-01-03 23:00:00,6141.6916,2754.9271,NaN,NaN,NaN
2011-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8885.84,8870.0,2011-01-04 23:00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-14,NaN,NaN,23.996667,24.0,24.0,11.647222,23.996667,23.996667,23.996667,0.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2020-02-15,NaN,NaN,23.997500,24.0,24.0,12.333333,23.997500,23.997500,11.664167,0.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2020-02-16,NaN,NaN,23.997500,24.0,24.0,23.997500,23.997500,23.997500,12.333333,0.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
